# Results of LeNet Trained on MNIST

In this notebook I display results in terms of accuracy on test data for LeNet models trained on the MNIST dataset using various regularization techniques. The LeNet model and the MNIST dataset were set up as in Hoffman 2019 (for all the models). The MNIST data is preprocessed by normalizing using mean 0.1307 and variance 0.3081. The batch size is 100. The model optimizes using SGD with momentum p = 0.9, and standard cross-entropy loss. Model parameters are initialized using Glorot initialization (See Glorot & Bengio 2010), expect for SVB regularization which uses orthogonal initialization. Models are trained with no regularization, L2 regularization, SVB regularization and Jacobian regularization, both with and without dropout with a dropout rate of p_drop = 0.5. The L2 regularization coefficient and Jacobian regularization coefficient are the same as in Hoffman 2019: l2_lmbd = 0.0005 and lambda_jacobian_reg = 0.01. For SVB regularization I use the hyperparameters svb_freq=600 and svb_eps = 0.05. The learning rate starts at 0.1, and is reduced to 0.01 and 0.001 1/3 and 2/3s into training, respectively. The models are trained for 250 epochs.

### Imports and Model Loading

In [1]:
import jupyter_black
import numpy as np
import pandas as pd
import torch
from scipy import stats
from torchsummary import summary
from tqdm import tqdm

from data_generators import data_loader_MNIST
from model_classes import LeNet_MNIST
from plotting_tools import get_random_img, generate_random_vectors
from tools import accuracy, compute_total_variation, ModelInfo

jupyter_black.load()

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MNIST data
train_loader, test_loader = data_loader_MNIST()

# Summary of model
summary_model = LeNet_MNIST().to(device)
summary(summary_model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
         MaxPool2d-2            [-1, 6, 14, 14]               0
            Conv2d-3           [-1, 16, 10, 10]           2,416
         MaxPool2d-4             [-1, 16, 5, 5]               0
            Linear-5                  [-1, 120]          48,120
           Dropout-6                  [-1, 120]               0
            Linear-7                   [-1, 84]          10,164
           Dropout-8                   [-1, 84]               0
            Linear-9                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.24
Estimated Total Size (MB): 0.30
---------------------------------------------

In [3]:
# Load models

# Defining model types and versions
model_types = ["no_reg", "l2", "jacobi", "svb"]
model_versions = ["", "_no_dropout"]

model_names = []

for t in model_types:
    for v in model_versions:
        for i in range(5):
            model_names.append(f"model_{t}{v}_{i}")

models = {name: ModelInfo(name) for name in model_names}

### Accuracies

Here I calculate the accuracies of each model as a 95% confidence interval. I have trained five models of each variation that I use to calculate the CI.

In [4]:
# Dictionary to hold accuracy data
accuracy_data = {}

# Calculate accuracies with 95% CI for models
for t in model_types:
    for v in model_versions:
        accuracies = []
        for i in range(5):  # Assuming 5 versions of each model
            model_name = f"model_{t}{v}_{i}"
            model = models[model_name].model
            acc = accuracy(model, test_loader, device)
            accuracies.append(acc)

        # Calculate mean accuracy
        mean_accuracy = np.mean(accuracies)

        # Calculate standard error
        std_error = stats.sem(accuracies)

        # Calculate confidence interval
        CI = std_error * stats.t.ppf((1 + 0.95) / 2, len(accuracies) - 1)

        # Store results in the data dictionary
        accuracy_data[f"{t}{v if v != '' else 'dropout'}"] = (mean_accuracy, CI)

In [14]:
# Convert the dictionary into a DataFrame
df = pd.DataFrame.from_dict(
    accuracy_data, orient="index", columns=["Mean Accuracy", "Confidence Interval"]
)

# Convert the numbers from decimal to percentage and round to four decimal places
df = df.multiply(100).round(3)

# Create a new column that combines Mean Accuracy and Confidence Interval
df["Accuracy +/- CI (%)"] = df.apply(
    lambda row: f"{row['Mean Accuracy']} +/- {row['Confidence Interval']}", axis=1
)

# Drop the original columns
df = df.drop(columns=["Mean Accuracy", "Confidence Interval"])

# Reset the index
df = df.reset_index().rename(columns={"index": "Model"})

display(df)

,Model,Accuracy +/- CI (%)
0,no_reg_dropout,98.684 +/- 0.0743
1,no_reg__no_dropout,98.872 +/- 0.0875
2,l2_dropout,99.144 +/- 0.0867
3,l2__no_dropout,99.162 +/- 0.0592
4,jacobi_dropout,98.652 +/- 0.1105
5,jacobi__no_dropout,98.846 +/- 0.0867
6,svb_dropout,98.416 +/- 0.0822
7,svb__no_dropout,98.716 +/- 0.0524


### Total Variation

Total variation is the sum of the absolute differences for neighboring pixel-values in the input images. This measures how much noise is in the images. I generate 25 different images for each model to get a good mean and standard deviation, and give the results as a tabel for each model with the eight models (models with and without dropout) as rows, and the three different zoom levels as columns. The tabel contains the mean and 95 % confidence interval for the total variation for each model at each zoom level.

In [33]:
# Define zoom levels, number of images, and confidence level for confidence interval calculation
zoom_levels = [0.025, 0.01, 0.001]
n_images = 2
confidence_level = 0.95

# Select only the first model of each type (those with index 0)
selected_models = [name for name in model_names if name.split("_")[-1] == "0"]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results = pd.DataFrame(index=selected_models, columns=cols)

# Loop over the selected models
for name in tqdm(selected_models):
    model = models[name].model  # Get model
    model.to(device)
    tv_values = {
        zoom: [] for zoom in zoom_levels
    }  # To store total variation values for each zoom level

    # Generate tv values for n_images number of images
    for _ in range(n_images):
        # Get random image and vectors
        img = get_random_img(test_loader)
        v1, v2 = generate_random_vectors(img)

        # Compute the total variation of decision boundaries for the current model
        # and the generated image at different zoom levels
        tv_list = compute_total_variation(model, img, v1, v2, device)
        for zoom, tv in zip(zoom_levels, tv_list):
            tv_values[zoom].append(tv)

    # Calculate mean and 95% confidence interval for total variation values at each zoom level
    for zoom in zoom_levels:
        mean = np.mean(tv_values[zoom])
        std_err = np.std(tv_values[zoom]) / np.sqrt(n_images)
        conf_interval = stats.t.ppf((1 + confidence_level) / 2, n_images - 1) * std_err
        df_results.loc[name, (zoom, "mean")] = mean
        df_results.loc[name, (zoom, "conf_interval")] = conf_interval

100%|██████████| 8/8 [03:52<00:00, 29.02s/it]


In [34]:
display(df_results)

0.025                  0.010                \
                              mean conf_interval     mean conf_interval   
model_no_reg_0             17914.0  86027.961822  20942.5  29833.508849   
model_no_reg_no_dropout_0   6402.0  55543.066317  13024.0  12722.255242   
model_l2_0                  6044.0   10530.00222   4975.0   3477.057047   
model_l2_no_dropout_0       1051.0    6639.65157   3717.5   2448.315363   
model_jacobi_0             17690.0  59981.480221  18221.0  41527.022406   
model_jacobi_no_dropout_0   3127.0    1069.17258   2942.0  14968.416125   
model_svb_0                 2260.5  20309.786705   3320.0   8131.102397   
model_svb_no_dropout_0      1281.0  11509.328365   3742.0   1536.374044   

                             0.001                
                              mean conf_interval  
model_no_reg_0             30045.0  21706.898774  
model_no_reg_no_dropout_0  36326.0   7430.300201  
model_l2_0                  8174.0   8158.056327  
model_l2_no_dropout_0       8241.5  16149.896749  
model_jacobi_0             15791.0  40457.849826  
model_jacobi_no_dropout_0   8011.5  17767.132585  
model_svb_0                 5745.0   5327.893615  
model_svb_no_dropout_0     11491.0   8867.843166